Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [Dels.com](https://dels.com/locations/) | [HERE API](https://developer.here.com/).
<hr>

# Del's Lemonade Isochrone Map

Drive time isochrones to the best summer time treat in Rhode Island Del's Lemonade. This notebook will use the HERE API to get the drive time from all locations on the companies website. Del's provides location data on their website with google maps which provides KML files. The data is combined using geopandas and shapely later the geojson will be combined in QGIS and stylized and finally edited in Photoshop.
<hr>

In [1]:
import json
import time
import requests
import pandas as pd
from pathlib import Path

import flexpolyline
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
def isoline_urls(df, time_range, tranport):
    coded_range = ','.join(str(x) for x in time_range)
    
    urls = {}
    for key, lat, lng in zip(df['key'], df['fields.lat'], df['fields.lng']):
        url = "https://isoline.router.hereapi.com/v8/isolines?" + \
             f"apiKey={api_key}" + \
             f"&range[type]=time" + \
             f"&range[values]={coded_range}" + \
             f"&transportMode={transport_type}" + \
             f"&destination={lat},{lng}"

        urls[key] = url
        
    return urls

def isoline_response(key, url):
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
    
    cols = ['id','store_key', 'location_lng', 'location_lat', 'seconds', 'geometry']
    gdf = gpd.GeoDataFrame(columns=cols, crs="EPSG:4326")
    lat = data['arrival']['place']['originalLocation']['lat']
    lng = data['arrival']['place']['originalLocation']['lng']
    
    for i, poly in enumerate(data['isolines']):
        seconds = (poly['range']['value'])
        polygon = poly['polygons'][0]['outer']
        geom = flexpolyline.decode(polygon)
        reverse_geom = [(xy[1], xy[0]) for xy in geom]
        gdf.loc[i] = [i, key, lng, lat, seconds, reverse_geom]
        
    return gdf

def collect_isolines(df, api_key, time_range, transport_type, sleep):
    # filter for already collected keys
    files = list(Path('./data/raw/').glob('*.geojson'))
    collected_keys = [x.stem for x in files]
    df = df[~df['key'].isin(collected_keys)]
    
    isolines = isoline_urls(df, time_range, transport_type)
    
    frames = []
    for key, url in isolines.items():
        gdf = isoline_response(key, url)
        gdf['geometry'] = gdf['geometry'].apply(Polygon)

        frames.append(gdf)
        gdf.to_file(f"./data/raw/{key}.geojson", driver='GeoJSON')
        time.sleep(sleep)
    
    return pd.concat(frames)

In [3]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file('./data/files/dels-locations.kml', driver='KML')
df = df[['Name', 'geometry']]

df = (df
    .rename(columns={'Name': 'key'})
    .assign(x = df['geometry'].x)
    .assign(y = df['geometry'].y)
    .drop(columns = 'geometry')
    .rename(columns={'x': 'fields.lng', 'y': 'fields.lat'})
)

In [4]:
with open('./config.json', 'r') as f:
    config = json.load(f)
    
api_key = config['api_key']

time_range = [150, 300, 600, 900]
transport_type = 'car'

gdf = collect_isolines(df, api_key, time_range, transport_type, 5)
gdf.to_file("./data/clean/combined.geojson", driver='GeoJSON')